# Data Preparation

This Jupyter Notebook cleans & prepares the raw Steam Games dataset for further analysis.

In [17]:
# Import libraries
import numpy as np
import pandas as pd
import re

from datetime import datetime, timedelta
from dateutil.parser import parse
from sklearn.preprocessing import MultiLabelBinarizer

In [18]:
games = pd.read_csv('datasets/raw_steam_games.csv')
games

,url,types,name,desc_snippet,recent_reviews,all_reviews,release_date,developer,publisher,popular_tags,game_details,languages,achievements,genre,game_description,mature_content,minimum_requirements,recommended_requirements,original_price,discount_price
0,https://store.steampowered.com/app/379720/DOOM/,app,DOOM,Now includes all three premium DLC packs (Unto...,"Very Positive,(554),- 89% of the 554 user revi...","Very Positive,(42,550),- 92% of the 42,550 use...","May 12, 2016",id Software,"Bethesda Softworks,Bethesda Softworks","FPS,Gore,Action,Demons,Shooter,First-Person,Gr...","Single-player,Multi-player,Co-op,Steam Achieve...","English,French,Italian,German,Spanish - Spain,...",54.0,Action,"About This Game Developed by id software, the...",NaN,"Minimum:,OS:,Windows 7/8.1/10 (64-bit versions...","Recommended:,OS:,Windows 7/8.1/10 (64-bit vers...",$19.99,$14.99
1,https://store.steampowered.com/app/578080/PLAY...,app,PLAYERUNKNOWN'S BATTLEGROUNDS,PLAYERUNKNOWN'S BATTLEGROUNDS is a battle roya...,"Mixed,(6,214),- 49% of the 6,214 user reviews ...","Mixed,(836,608),- 49% of the 836,608 user revi...","Dec 21, 2017",PUBG Corporation,"PUBG Corporation,PUBG Corporation","Survival,Shooter,Multiplayer,Battle Royale,PvP...","Multi-player,Online Multi-Player,Stats","English,Korean,Simplified Chinese,French,Germa...",37.0,"Action,Adventure,Massively Multiplayer",About This Game PLAYERUNKNOWN'S BATTLEGROUND...,Mature Content Description The developers de...,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$29.99,NaN
2,https://store.steampowered.com/app/637090/BATT...,app,BATTLETECH,Take command of your own mercenary outfit of '...,"Mixed,(166),- 54% of the 166 user reviews in t...","Mostly Positive,(7,030),- 71% of the 7,030 use...","Apr 24, 2018",Harebrained Schemes,"Paradox Interactive,Paradox Interactive","Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","Single-player,Multi-player,Online Multi-Player...","English,French,German,Russian",128.0,"Action,Adventure,Strategy",About This Game From original BATTLETECH/Mec...,NaN,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$39.99,NaN
3,https://store.steampowered.com/app/221100/DayZ/,app,DayZ,The post-soviet country of Chernarus is struck...,"Mixed,(932),- 57% of the 932 user reviews in t...","Mixed,(167,115),- 61% of the 167,115 user revi...","Dec 13, 2018",Bohemia Interactive,"Bohemia Interactive,Bohemia Interactive","Survival,Zombies,Open World,Multiplayer,PvP,Ma...","Multi-player,Online Multi-Player,Steam Worksho...","English,French,Italian,German,Spanish - Spain,...",NaN,"Action,Adventure,Massively Multiplayer",About This Game The post-soviet country of Ch...,NaN,"Minimum:,OS:,Windows 7/8.1 64-bit,Processor:,I...","Recommended:,OS:,Windows 10 64-bit,Processor:,...",$44.99,NaN
4,https://store.steampowered.com/app/8500/EVE_On...,app,EVE Online,EVE Online is a community-driven spaceship MMO...,"Mixed,(287),- 54% of the 287 user reviews in t...","Mostly Positive,(11,481),- 74% of the 11,481 u...","May 6, 2003",CCP,"CCP,CCP","Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","Multi-player,Online Multi-Player,MMO,Co-op,Onl...","English,German,Russian,French",NaN,"Action,Free to Play,Massively Multiplayer,RPG,...",About This Game,NaN,"Minimum:,OS:,Windows 7,Processor:,Intel Dual C...","Recommended:,OS:,Windows 10,Processor:,Intel i...",Free,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40828,https://store.steampowered.com/app/899836/Rock...,app,Rocksmith® 2014 Edition – Remastered – Sabaton...,NaN,NaN,NaN,"Feb 12, 2019",Ubisoft - San Francisco,NaN,"Casual,Simulation","Single-player,Shared/Split Screen,Downloadable...","English,German,French,Italian,Spanish - Spain,...",NaN,"Casual,Simulation","About This Content Play ""Ghost Division"" by S...",NaN,"Minimum:,OS:,Windows Vista, Windows 7, Windows...","Recommended:,OS:,Windows Vista, Windows 7,

## Removing malformed rows

In [19]:
games = games[games['types'] == 'app'] # remove non-game rows
games = games.dropna(subset = ['all_reviews', 'release_date']) # remove games that do not have `all_review`/`release_date`, i.e. unreleased games
games = games.dropna(subset = ['genre', 'languages']) # remove malformed rows that do not include genres/languages
games = games[~games['all_reviews'].str.contains('Need more user reviews to generate a score')] # remove games that do not have enough user reviews yet
games

,url,types,name,desc_snippet,recent_reviews,all_reviews,release_date,developer,publisher,popular_tags,game_details,languages,achievements,genre,game_description,mature_content,minimum_requirements,recommended_requirements,original_price,discount_price
0,https://store.steampowered.com/app/379720/DOOM/,app,DOOM,Now includes all three premium DLC packs (Unto...,"Very Positive,(554),- 89% of the 554 user revi...","Very Positive,(42,550),- 92% of the 42,550 use...","May 12, 2016",id Software,"Bethesda Softworks,Bethesda Softworks","FPS,Gore,Action,Demons,Shooter,First-Person,Gr...","Single-player,Multi-player,Co-op,Steam Achieve...","English,French,Italian,German,Spanish - Spain,...",54.0,Action,"About This Game Developed by id software, the...",NaN,"Minimum:,OS:,Windows 7/8.1/10 (64-bit versions...","Recommended:,OS:,Windows 7/8.1/10 (64-bit vers...",$19.99,$14.99
1,https://store.steampowered.com/app/578080/PLAY...,app,PLAYERUNKNOWN'S BATTLEGROUNDS,PLAYERUNKNOWN'S BATTLEGROUNDS is a battle roya...,"Mixed,(6,214),- 49% of the 6,214 user reviews ...","Mixed,(836,608),- 49% of the 836,608 user revi...","Dec 21, 2017",PUBG Corporation,"PUBG Corporation,PUBG Corporation","Survival,Shooter,Multiplayer,Battle Royale,PvP...","Multi-player,Online Multi-Player,Stats","English,Korean,Simplified Chinese,French,Germa...",37.0,"Action,Adventure,Massively Multiplayer",About This Game PLAYERUNKNOWN'S BATTLEGROUND...,Mature Content Description The developers de...,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$29.99,NaN
2,https://store.steampowered.com/app/637090/BATT...,app,BATTLETECH,Take command of your own mercenary outfit of '...,"Mixed,(166),- 54% of the 166 user reviews in t...","Mostly Positive,(7,030),- 71% of the 7,030 use...","Apr 24, 2018",Harebrained Schemes,"Paradox Interactive,Paradox Interactive","Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","Single-player,Multi-player,Online Multi-Player...","English,French,German,Russian",128.0,"Action,Adventure,Strategy",About This Game From original BATTLETECH/Mec...,NaN,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$39.99,NaN
3,https://store.steampowered.com/app/221100/DayZ/,app,DayZ,The post-soviet country of Chernarus is struck...,"Mixed,(932),- 57% of the 932 user reviews in t...","Mixed,(167,115),- 61% of the 167,115 user revi...","Dec 13, 2018",Bohemia Interactive,"Bohemia Interactive,Bohemia Interactive","Survival,Zombies,Open World,Multiplayer,PvP,Ma...","Multi-player,Online Multi-Player,Steam Worksho...","English,French,Italian,German,Spanish - Spain,...",NaN,"Action,Adventure,Massively Multiplayer",About This Game The post-soviet country of Ch...,NaN,"Minimum:,OS:,Windows 7/8.1 64-bit,Processor:,I...","Recommended:,OS:,Windows 10 64-bit,Processor:,...",$44.99,NaN
4,https://store.steampowered.com/app/8500/EVE_On...,app,EVE Online,EVE Online is a community-driven spaceship MMO...,"Mixed,(287),- 54% of the 287 user reviews in t...","Mostly Positive,(11,481),- 74% of the 11,481 u...","May 6, 2003",CCP,"CCP,CCP","Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","Multi-player,Online Multi-Player,MMO,Co-op,Onl...","English,German,Russian,French",NaN,"Action,Free to Play,Massively Multiplayer,RPG,...",About This Game,NaN,"Minimum:,OS:,Windows 7,Processor:,Intel Dual C...","Recommended:,OS:,Windows 10,Processor:,Intel i...",Free,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40557,https://store.steampowered.com/app/652810/Grab...,app,Grabity,"Dance the tango of death in Grabity, a fast-pa...",NaN,"Positive,(17),- 100% of the 17 user reviews fo...","May 30, 2018",Team Ninja Thumbs,"Team Ninja Thumbs,Team Ninja Thumbs","Indie,Action,4 Player Local,Local Multiplayer,...","Single-player,Multi-player,Online Multi-Player...","English,German,French,Portuguese,Spanish - Spain",15.0,"Action,Indie",About This Game Welcome to Grabity's ballis

## Removing unused columns

In [20]:
# All these fields are either do not provide meaningful information or are too difficult to extract useful information from.
# TODO: drop popular_tags, languages, mature_content & discounted_price
games = games.drop(['url', 'types', 'desc_snippet', 'recent_reviews', 'game_description', 'game_details', 'minimum_requirements', 'recommended_requirements', 'achievements'], axis = 1)
games


,name,all_reviews,release_date,developer,publisher,popular_tags,languages,genre,mature_content,original_price,discount_price
0,DOOM,"Very Positive,(42,550),- 92% of the 42,550 use...","May 12, 2016",id Software,"Bethesda Softworks,Bethesda Softworks","FPS,Gore,Action,Demons,Shooter,First-Person,Gr...","English,French,Italian,German,Spanish - Spain,...",Action,NaN,$19.99,$14.99
1,PLAYERUNKNOWN'S BATTLEGROUNDS,"Mixed,(836,608),- 49% of the 836,608 user revi...","Dec 21, 2017",PUBG Corporation,"PUBG Corporation,PUBG Corporation","Survival,Shooter,Multiplayer,Battle Royale,PvP...","English,Korean,Simplified Chinese,French,Germa...","Action,Adventure,Massively Multiplayer",Mature Content Description The developers de...,$29.99,NaN
2,BATTLETECH,"Mostly Positive,(7,030),- 71% of the 7,030 use...","Apr 24, 2018",Harebrained Schemes,"Paradox Interactive,Paradox Interactive","Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","English,French,German,Russian","Action,Adventure,Strategy",NaN,$39.99,NaN
3,DayZ,"Mixed,(167,115),- 61% of the 167,115 user revi...","Dec 13, 2018",Bohemia Interactive,"Bohemia Interactive,Bohemia Interactive","Survival,Zombies,Open World,Multiplayer,PvP,Ma...","English,French,Italian,German,Spanish - Spain,...","Action,Adventure,Massively Multiplayer",NaN,$44.99,NaN
4,EVE Online,"Mostly Positive,(11,481),- 74% of the 11,481 u...","May 6, 2003",CCP,"CCP,CCP","Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","English,German,Russian,French","Action,Free to Play,Massively Multiplayer,RPG,...",NaN,Free,NaN
...,...,...,...,...,...,...,...,...,...,...,...
40557,Grabity,"Positive,(17),- 100% of the 17 user reviews fo...","May 30, 2018",Team Ninja Thumbs,"Team Ninja Thumbs,Team Ninja Thumbs","Indie,Action,4 Player Local,Local Multiplayer,...","English,German,French,Portuguese,Spanish - Spain","Action,Indie",NaN,Download Demo,NaN
40598,Detached: Non-VR Edition,"Mostly Positive,(14),- 78% of the 14 user revi...","Jul 24, 2018",Anshar Studios,"Anshar Studios,Anshar Studios","Simulation,Indie,Space,Exploration,Atmospheric...","English,French,German,Simplified Chinese,Polish","Indie,Simulation",NaN,$14.99,NaN
40611,A Room Beyond,"Positive,(11),- 90% of the 11 user reviews for...","Jun 13, 2017",René Bühling,"René Bühling,René Bühling","Adventure,Indie,Point & Click","English,German,French,Italian,Spanish - Spain","Adventure,Indie",NaN,$7.99,NaN
40728,Chasing the Stars,"Mostly Positive,(10),- 70% of the 10 user revi...","Jan 23, 2019",Ertal Games,"Ertal Games,Ertal Games","Nudity,Sexual Content,Indie,Steampunk,Story Ri...",English,Indie,Mature Content Description The developers de...,$9.99,NaN


## Clean `original_price` column

In [21]:
pattern = re.compile(r'demo|trial|try|prototype|prologue|(^(?=.*\bfree\b)(?=.*\bedition\b).*$)', re.IGNORECASE) # We try our best to filter out trial/demo versions of a game
mask = games['original_price'].str.contains(pattern, regex=True, na=True)

games = games[~mask]
games

C:\Users\Matthias\AppData\Local\Temp\ipykernel_12736\2606443298.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = games['original_price'].str.contains(pattern, regex=True, na=True)


,name,all_reviews,release_date,developer,publisher,popular_tags,languages,genre,mature_content,original_price,discount_price
0,DOOM,"Very Positive,(42,550),- 92% of the 42,550 use...","May 12, 2016",id Software,"Bethesda Softworks,Bethesda Softworks","FPS,Gore,Action,Demons,Shooter,First-Person,Gr...","English,French,Italian,German,Spanish - Spain,...",Action,NaN,$19.99,$14.99
1,PLAYERUNKNOWN'S BATTLEGROUNDS,"Mixed,(836,608),- 49% of the 836,608 user revi...","Dec 21, 2017",PUBG Corporation,"PUBG Corporation,PUBG Corporation","Survival,Shooter,Multiplayer,Battle Royale,PvP...","English,Korean,Simplified Chinese,French,Germa...","Action,Adventure,Massively Multiplayer",Mature Content Description The developers de...,$29.99,NaN
2,BATTLETECH,"Mostly Positive,(7,030),- 71% of the 7,030 use...","Apr 24, 2018",Harebrained Schemes,"Paradox Interactive,Paradox Interactive","Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","English,French,German,Russian","Action,Adventure,Strategy",NaN,$39.99,NaN
3,DayZ,"Mixed,(167,115),- 61% of the 167,115 user revi...","Dec 13, 2018",Bohemia Interactive,"Bohemia Interactive,Bohemia Interactive","Survival,Zombies,Open World,Multiplayer,PvP,Ma...","English,French,Italian,German,Spanish - Spain,...","Action,Adventure,Massively Multiplayer",NaN,$44.99,NaN
4,EVE Online,"Mostly Positive,(11,481),- 74% of the 11,481 u...","May 6, 2003",CCP,"CCP,CCP","Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","English,German,Russian,French","Action,Free to Play,Massively Multiplayer,RPG,...",NaN,Free,NaN
...,...,...,...,...,...,...,...,...,...,...,...
40524,Lil Big Invasion,"Positive,(13),- 100% of the 13 user reviews fo...","Aug 4, 2016",Andreas Britten,"Andreas Britten,Andreas Britten","Action,Indie,Adventure,Puzzle,Atmospheric,Cute...",English,"Action,Adventure,Indie",NaN,$2.99,NaN
40598,Detached: Non-VR Edition,"Mostly Positive,(14),- 78% of the 14 user revi...","Jul 24, 2018",Anshar Studios,"Anshar Studios,Anshar Studios","Simulation,Indie,Space,Exploration,Atmospheric...","English,French,German,Simplified Chinese,Polish","Indie,Simulation",NaN,$14.99,NaN
40611,A Room Beyond,"Positive,(11),- 90% of the 11 user reviews for...","Jun 13, 2017",René Bühling,"René Bühling,René Bühling","Adventure,Indie,Point & Click","English,German,French,Italian,Spanish - Spain","Adventure,Indie",NaN,$7.99,NaN
40728,Chasing the Stars,"Mostly Positive,(10),- 70% of the 10 user revi...","Jan 23, 2019",Ertal Games,"Ertal Games,Ertal Games","Nudity,Sexual Content,Indie,Steampunk,Story Ri...",English,Indie,Mature Content Description The developers de...,$9.99,NaN


## Clean `genres` column

In [22]:
others = [
    'Photo Editing', 'Video Production', 'Animation & Modeling', 'Utilities', 'Web Publishing', 'Software Training', 'Game Development', 'Audio Production', 'Accounting', 'Design & Illustration', 'Movie', 'Education', # non-games
    'Valve', # There's only one row, and it's for the steam controller,
    'Free to Play', # This genre is already represented by the `original_price` column
    'Early Access', 'Indie', # These aren't really "genre"s per-say especially when compared to the other genres
]

games.loc[:, 'genre'] = games['genre'].apply(lambda x: [] if pd.isna(x) else set(genre for genre in x.split(',') if genre not in others))

mask = games['genre'].apply(lambda x: len(x) == 0) # Remove any rows without genres, this usually means that the row isn't a game (Steam apparently sells game engines & other development tools)
games = games[~mask]

binarizer = MultiLabelBinarizer(sparse_output = True)

genres = pd.DataFrame.sparse.from_spmatrix(binarizer.fit_transform(games['genre']), index=games.index, columns=binarizer.classes_)
genres = genres.add_prefix(f'genre_')

games = pd.concat([games, genres], axis=1)
games

C:\Users\Matthias\AppData\Local\Temp\ipykernel_12736\4160768445.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games.loc[:, 'genre'] = games['genre'].apply(lambda x: [] if pd.isna(x) else set(genre for genre in x.split(',') if genre not in others))


,name,all_reviews,release_date,developer,publisher,popular_tags,languages,genre,mature_content,original_price,discount_price,genre_Action,genre_Adventure,genre_Casual,genre_Massively Multiplayer,genre_RPG,genre_Racing,genre_Simulation,genre_Sports,genre_Strategy
0,DOOM,"Very Positive,(42,550),- 92% of the 42,550 use...","May 12, 2016",id Software,"Bethesda Softworks,Bethesda Softworks","FPS,Gore,Action,Demons,Shooter,First-Person,Gr...","English,French,Italian,German,Spanish - Spain,...",{Action},NaN,$19.99,$14.99,1,0,0,0,0,0,0,0,0
1,PLAYERUNKNOWN'S BATTLEGROUNDS,"Mixed,(836,608),- 49% of the 836,608 user revi...","Dec 21, 2017",PUBG Corporation,"PUBG Corporation,PUBG Corporation","Survival,Shooter,Multiplayer,Battle Royale,PvP...","English,Korean,Simplified Chinese,French,Germa...","{Action, Adventure, Massively Multiplayer}",Mature Content Description The developers de...,$29.99,NaN,1,1,0,1,0,0,0,0,0
2,BATTLETECH,"Mostly Positive,(7,030),- 71% of the 7,030 use...","Apr 24, 2018",Harebrained Schemes,"Paradox Interactive,Paradox Interactive","Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","English,French,German,Russian","{Action, Adventure, Strategy}",NaN,$39.99,NaN,1,1,0,0,0,0,0,0,1
3,DayZ,"Mixed,(167,115),- 61% of the 167,115 user revi...","Dec 13, 2018",Bohemia Interactive,"Bohemia Interactive,Bohemia Interactive","Survival,Zombies,Open World,Multiplayer,PvP,Ma...","English,French,Italian,German,Spanish - Spain,...","{Action, Adventure, Massively Multiplayer}",NaN,$44.99,NaN,1,1,0,1,0,0,0,0,0
4,EVE Online,"Mostly Positive,(11,481),- 74% of the 11,481 u...","May 6, 2003",CCP,"CCP,CCP","Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","English,German,Russian,French","{Action, Strategy, RPG, Massively Multiplayer}",NaN,Free,NaN,1,0,0,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40481,雨上がりのハナビィ Ameagari no Hanaby,"Positive,(30),- 100% of the 30 user reviews fo...","Sep 22, 2018",Enigmatic Network,"Enigmatic Network,Enigmatic Network","Indie,Adventure,Action,Casual,Sexual Content,Cute","English,Japanese","{Action, Adventure, Casual}",Mature Content Description The developers de...,$13.99,NaN,1,1,1,0,0,0,0,0,0
40524,Lil Big Invasion,"Positive,(13),- 100% of the 13 user reviews fo...","Aug 4, 2016",Andreas Britten,"Andreas Britten,Andreas Britten","Action,Indie,Adventure,Puzzle,Atmospheric,Cute...",English,"{Action, Adventure}",NaN,$2.99,NaN,1,1,0,0,0,0,0,0,0
40598,Detached: Non-VR Edition,"Mostly Positive,(14),- 78% of the 14 user revi...","Jul 24, 2018",Anshar Studios,"Anshar Studios,Anshar Studios","Simulation,Indie,Space,Exploration,Atmospheric...","English,French,German,Simplified Chinese,Polish",{Simulation},NaN,$14.99,NaN,0,0,0,0,0,0,1,0,0
40611,A Room Beyond,"Positive,(11),- 90% of the 11 user reviews for...","Jun 13, 2017",René Bühling,"René Bühling,René Bühling","Adventure,Indie,Point & Click","English,German,French,Italian,Spanish - Spain",{Adventure},NaN,$7.99,NaN,0,1,0,0,0,0,0,0,0


## Cleaning other data

In [23]:
def map_all_reviews(x):
    match = re.search(r"\d{1,3}%", x)
    if match:
        return match.group(0)[:-1]

def map_original_price(x):
    """
    Transforms the given price's string representation to a numerical value. Any string that is not a price, i.e. "Play for free" is converted to 0.
    :param x: a price's string representation
    :return: a numerical price
    """
    if pd.isna(x):
        return x
    elif re.match(r'(\$\d+(\.\d{2})?)', x):
        return x[1:]
    else:
        return 0.0

def one_hot_encode(df, column_name, prefix=None):
    if prefix is None:
        prefix = column_name

    df[column_name] = df[column_name].apply(lambda x: [] if pd.isna(x) else list(x.split(',')))
    binarizer = MultiLabelBinarizer(sparse_output = True)

    tags = pd.DataFrame.sparse.from_spmatrix(binarizer.fit_transform(df.pop(column_name)), index=df.index, columns=binarizer.classes_)
    tags = tags.add_prefix(f'{prefix}_')

    df = pd.concat([df, tags], axis=1)
    return df

games['all_reviews'] = games['all_reviews'].apply(map_all_reviews)
games['release_date_epoch_days'] = games['release_date'].apply(lambda x: (parse(x) - datetime(1970, 1, 1)).days)
games['release_date'] = games['release_date'].apply(lambda x: parse(x).strftime('%Y-%m-%d'))
games = one_hot_encode(games, 'popular_tags')
games = one_hot_encode(games, 'languages')
games['mature_content'] = games['mature_content'].apply(lambda x: not pd.isna(x))
games['original_price'] = games['original_price'].apply(map_original_price)
games['discount_price'] = games['discount_price'].apply(lambda x: x if pd.isna(x) else x[1:])
games['publisher'] = games['publisher'].str.split(',').str.get(0)
games['all_reviews'] = games['all_reviews'].astype(int)
games['release_date'] = pd.to_datetime(games['release_date'])


games

,name,all_reviews,release_date,developer,publisher,genre,mature_content,original_price,discount_price,genre_Action,...,languages_Simplified Chinese,languages_Slovakian,languages_Spanish - Latin America,languages_Spanish - Spain,languages_Swedish,languages_Thai,languages_Traditional Chinese,languages_Turkish,languages_Ukrainian,languages_Vietnamese
0,DOOM,92,2016-05-12,id Software,"Bethesda Softworks,Bethesda Softworks",{Action},False,19.99,14.99,1,...,0,0,0,1,0,0,1,0,0,0
1,PLAYERUNKNOWN'S BATTLEGROUNDS,49,2017-12-21,PUBG Corporation,"PUBG Corporation,PUBG Corporation","{Action, Adventure, Massively Multiplayer}",True,29.99,NaN,1,...,1,0,0,1,0,1,1,1,1,0
2,BATTLETECH,71,2018-04-24,Harebrained Schemes,"Paradox Interactive,Paradox Interactive","{Action, Adventure, Strategy}",False,39.99,NaN,1,...,0,0,0,0,0,0,0,0,0,0
3,DayZ,61,2018-12-13,Bohemia Interactive,"Bohemia Interactive,Bohemia Interactive","{Action, Adventure, Massively Multiplayer}",False,44.99,NaN,1,...,1,0,0,1,0,0,1,0,0,0
4,EVE Online,74,2003-05-06,CCP,"CCP,CCP","{Action, Strategy, RPG, Massively Multiplayer}",False,0.0,NaN,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40481,雨上がりのハナビィ Ameagari no Hanaby,100,2018-09-22,Enigmatic Network,"Enigmatic Network,Enigmatic Network","{Action, Adventure, Casual}",True,13.99,NaN,1,...,0,0,0,0,0,0,0,0,0,0
40524,Lil Big Invasion,100,2016-08-04,Andreas Britten,"Andreas Britten,Andreas Britten","{Action, Adventure}",False,2.99,NaN,1,...,0,0,0,0,0,0,0,0,0,0
40598,Detached: Non-VR Edition,78,2018-07-24,Anshar Studios,"Anshar Studios,Anshar Studios",{Simulation},False,14.99,NaN,0,...,1,0,0,0,0,0,0,0,0,0
40611,A Room Beyond,90,2017-06-13,René Bühling,"René Bühling,René Bühling",{Adventure},False,7.99,NaN,0,...,0,0,0,1,0,0,0,0,0,0


C:\Users\Isaac\AppData\Local\Temp\ipykernel_1916\611468486.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games['discount_percentage'] = (games['original_price'] - games['discount_price'])/games['original_price'] * 100
C:\Users\Isaac\AppData\Local\Temp\ipykernel_1916\611468486.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games['discount_percentage'] = games['discount_percentage'].fillna(0)


In [24]:
# We store the results as Pickle instead of CSV since it allows us to preserve data structures, i.e. sparse lists.
games.to_pickle('datasets/cleaned_steam_games.pkl')

# To read the file, `df = pd.read_pickle('datasets/cleaned_steam_games.pkl')`